In [1]:
pip install lightgbm holidays

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.6 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730984 sha256=0a0df72117c57ae339ad621e22a08e8c736f4d21d2dd1b72c1955048fd945a96
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a0/8b/b2/810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import holidays
import time

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

## Basic feature engineering 
train['weekday'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['weekend'] = np.where(train['weekday'] >= 5, 1, 0)
train['dayOfMonth'] = train['date'].dt.day
train['dayOfYear'] = train['date'].dt.dayofyear

test['weekday'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['weekend'] = np.where(test['weekday'] >= 5, 1, 0)
test['dayOfMonth'] = test['date'].dt.day
test['dayOfYear'] = test['date'].dt.dayofyear

In [ ]:
from sklearn.preprocessing import LabelEncoder

all_data = pd.concat([train, test], axis = 0)

le = LabelEncoder()
cols = ['country', 'store', 'product']
for col in cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col])

In [ ]:
all_data = all_data.drop(["date","row_id"], axis = 1)
train = all_data.iloc[:70128,:]
test = all_data.iloc[70128:,:].drop(["num_sold"], axis = 1)

X = train.drop(['num_sold'], axis = 1)
Y = np.log(train['num_sold'])

# Belgium

In [12]:
data_temp = train[(train['country'] == 'Belgium') & (train['store'] == 'KaggleMart') & (train['product'] == 'Kaggle Advanced Techniques')].reset_index(drop = True)
data_temp.head()

,row_id,date,country,store,product,num_sold,weekday,month,weekend,dayOfMonth,dayOfYear
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,6,1,1,1,1
1,48,2017-01-02,Belgium,KaggleMart,Kaggle Advanced Techniques,514,0,1,0,2,2
2,96,2017-01-03,Belgium,KaggleMart,Kaggle Advanced Techniques,549,1,1,0,3,3
3,144,2017-01-04,Belgium,KaggleMart,Kaggle Advanced Techniques,477,2,1,0,4,4
4,192,2017-01-05,Belgium,KaggleMart,Kaggle Advanced Techniques,447,3,1,0,5,5


In [15]:
data_temp = train[(train['country'] == 'Belgium') & (train['store'] == 'KaggleMart') & (train['product'] == 'Kaggle Advanced Techniques')].reset_index(drop = True)

X = data_temp.drop(columns = ['row_id', 'date', 'country', 'store', 'product', 'num_sold'], axis = 1)
Y = np.log(data_temp['num_sold'])


t1 = time.time()
kf = KFold(n_splits = 5, shuffle = True, random_state = 841)
off = np.zeros(len(train))
score_list_lgb = []
test_preds_lgb = []
fold = 1

for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape, "X_val shape is", X_val.shape)
    y_pred_list = []
    
    model_lgb = LGBMRegressor(n_estimators = 3000, 
                              learning_rate = 0.059,
                              num_leaves = 37,
                              max_depth = 9, 
                              lambda_l1 = 3.69, 
                              lambda_l2 = 1.0152938, 
                              bagging_fraction = 0.95, 
                              feature_fraction = 0.96)

    model = model_lgb.fit(X_train, Y_train)
    result = model_lgb.predict(X_val)
    
    result = pd.DataFrame(result)
    result.iloc[:,0] = [0 if i <= 0 else i for i in result.iloc[:,0]]
    
    score = np.sqrt(mean_squared_error(Y_val, result))
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_lgb.append(score)

#     test_preds_lgb.append(model_lgb.predict(test))
    fold +=1

t2 = time.time()
print("LGBM model with cross validation take : {:.3f} sn.".format(t2-t1))

X_train shape is : (1168, 5) X_val shape is (293, 5)
[LightGBM] [Warning] lambda_l1 is set=3.69, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.69
[LightGBM] [Warning] feature_fraction is set=0.96, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.96
[LightGBM] [Warning] lambda_l2 is set=1.0152938, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0152938
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
Fold  1  result is: 0.10645964396923221 

X_train shape is : (1169, 5) X_val shape is (292, 5)
[LightGBM] [Warning] lambda_l1 is set=3.69, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.69
[LightGBM] [Warning] feature_fraction is set=0.96, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.96
[LightGBM] [Warning] lambda_l2 is set=1.0152938, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0152938
[LightGBM] [Warning] bagging_fraction i

In [16]:
mean = sum(score_list_lgb) / len(score_list_lgb)
variance = sum([((x - mean) ** 2) for x in score_list_lgb]) / len(score_list_lgb)
res = variance ** 0.5
print("Cross validation mean score:", sum(score_list_lgb) / len(score_list_lgb))
print("Cross validation score's Standart deviation is:", res)

Cross validation mean score: 0.10364315711272573
Cross validation score's Standart deviation is: 0.0017065428435641093


In [7]:
train.head()

,row_id,date,country,store,product,num_sold,weekday,month,weekend,dayOfMonth,dayOfYear
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,6,January,1,1,1
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,6,January,1,1,1
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,6,January,1,1,1
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,6,January,1,1,1
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,6,January,1,1,1
